In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [4]:
def GetWhiskers(column):
    times = 1.5
    q1q2q3 = np.quantile(column, [0.25,0.50,0.75])
    IQR = q1q2q3[2]-q1q2q3[0]
    w1 = q1q2q3[0]-times*IQR
    w2 = q1q2q3[2]+times*IQR
    return column[column > w1].min(), column[column < w2].max()

Разбиваем данные и целевые переменные на тренеровочные данные и проверочные в соотношении 7/10 (3/10 на проверку). Фиксируем random_state=1

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [6]:
clr = GaussianNB()
clr.fit(X_train, y_train)
y_pred = clr.predict(X_test)

In [7]:
accuracy_score(y_test, y_pred)

0.9333333333333333

In [8]:
col_names = iris.feature_names

In [9]:
df = pd.DataFrame(X_train, columns=col_names)

In [10]:
from sklearn.neighbors import LocalOutlierFactor

In [11]:
clf = LocalOutlierFactor(n_neighbors=5)
indexes = clf.fit_predict(df) > 0

In [12]:
df.nunique()

sepal length (cm)    31
sepal width (cm)     20
petal length (cm)    39
petal width (cm)     22
dtype: int64

In [13]:
dubles = df.duplicated()

In [14]:
df[dubles]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
93,5.8,2.7,5.1,1.9


In [15]:
df['y'] = y_train

In [16]:
#df.drop_duplicates(inplace=True)
#df = df[indexes]

In [17]:
whiskers = GetWhiskers(df['sepal width (cm)'])
print(whiskers)

(2.2, 3.8)


In [18]:
df = df[df['sepal width (cm)'] > whiskers[0]]
df = df[df['sepal width (cm)'] < whiskers[1]]

In [19]:
#df.loc[df['sepal width (cm)'] < whiskers[0], 'sepal width (cm)'] = whiskers[0]
#df.loc[df['sepal width (cm)'] > whiskers[1], 'sepal width (cm)'] = whiskers[1]

In [20]:
X_train2 = df[col_names]

In [21]:
y_train2 = df['y']

In [22]:
clr2 = GaussianNB()
clr2.fit(X_train2, y_train2)

GaussianNB(priors=None, var_smoothing=1e-09)

In [23]:
y_pred = clr2.predict(X_test)
print(y_pred)

[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 2 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 1 2 1]


In [24]:
accuracy_score(y_test, y_pred)

0.9555555555555556

In [25]:
def circ(i):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    #X_train = np.append(X_train, [[5.4, 0, 1.2, 0.3]], axis=0)
    #X_train = np.append(X_train, [[5.3, 0, 1.3,0.2]], axis=0)
    #X_train = np.append(X_train, [[5.5, 0, 1.1,0.3]], axis=0)
    #y_train = np.append(y_train, [0])
    clr = GaussianNB()
    #clr = DecisionTreeClassifier(max_depth=2)
    clr.fit(X_train, y_train)
    y_pred = clr.predict(X_test)
    acc1 = accuracy_score(y_test, y_pred)
    df = pd.DataFrame(X_train, columns=col_names)
    df['y'] = y_train
    whiskers = GetWhiskers(df['sepal width (cm)'])
    df = df[df['sepal width (cm)'] > whiskers[0]]
    df = df[df['sepal width (cm)'] < whiskers[1]]
    #df.loc[df['sepal width (cm)'] < whiskers[0], 'sepal width (cm)'] = whiskers[0]
    #df.loc[df['sepal width (cm)'] > whiskers[1], 'sepal width (cm)'] = whiskers[1]
    X_train2 = df[col_names]
    y_train2 = df['y']
    clr2 = GaussianNB()
    #clr2 = DecisionTreeClassifier(max_depth=2)
    clr2.fit(X_train2, y_train2)
    y_pred2 = clr2.predict(X_test)
    acc2 = accuracy_score(y_test, y_pred2)
    return acc1, acc2

In [27]:
for i in range(1, 100):
    cc = circ(i)
    if(cc[1]<cc[0]):
        print(i, cc)

3 (0.9777777777777777, 0.9555555555555556)
61 (0.9777777777777777, 0.9555555555555556)
82 (0.9333333333333333, 0.9111111111111111)
93 (0.9555555555555556, 0.9333333333333333)
